In [1]:
## DO NOT RUN THIS.  It is used to build an updated BkAvgAst... table which is already in the processing zip file.
## first step in creating new BkAvgAst... file
## 2017/04/07:  updated for Mar 2017 failure data (Note: only failures before 2017 used since Call data as of Dec-2016)
## 2018/03/08: updated for Dec 2017 failure events (since Call Data only up to Dec17)
## 2018/08/27: updated failure rate details (number fails/number comm bks)
import numpy as np
import pandas as pd
from datetime import datetime
#import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

import os
os.chdir('C:/BankRisk')

df2 = pd.read_csv("df2_2001_202003.csv")
df2['AsOf'] = pd.to_datetime(df2['AsOf'])

In [2]:
fcol = df2.columns[5:]  ## get listing of numeric fields, then...
df2[fcol] = df2[fcol].astype(float)  ### CRITICAL to convert numeric data to FLOAT!!! (since some are ints)

In [3]:
df3 = df2.sort_index(by=['IDRSSD','AsOf'], ascending=True).reset_index(drop=True)
df3.head()

Unnamed: 0  IDRSSD       AsOf                    Name State     Cert  \
0           0      37 2001-03-31  BANK OF HANCOCK COUNTY    GA  10057.0   
1        8857      37 2001-06-30  BANK OF HANCOCK COUNTY    GA  10057.0   
2       17647      37 2001-09-30  BANK OF HANCOCK COUNTY    GA  10057.0   
3       26405      37 2001-12-31  BANK OF HANCOCK COUNTY    GA  10057.0   
4       35094      37 2002-03-31  BANK OF HANCOCK COUNTY    GA  10057.0   

   eTotAst   eTotLn   eALLL  eOREO  ...  ncoFIfor  ncoAg  ncoCIdom  ncoCIfor  \
0  65486.0  40507.0   979.0  200.0  ...       0.0    0.0       0.0       0.0   
1  65427.0  42153.0  1049.0    0.0  ...       0.0    0.0       0.0       0.0   
2  65575.0  44359.0  1070.0   40.0  ...       0.0    0.0       0.0       0.0   
3  66624.0  41514.0   987.0   20.0  ...       0.0    0.0       5.0       0.0   
4  68766.0  41539.0  1048.0   20.0  ...       0.0    0.0       0.0       0.0   

   ncoCard  ncoAuto  ncoOthCsmr  ncoForGovt  ncoOthLn  ncoLeas  
0      0.0      0.0        -1.0         0.0       0.0      0.0  
1      0.0      0.0       -23.0         0.0       0.0      0.0  
2      0.0      0.0         2.0         0.0       0.0      0.0  
3      0.0      0.0       104.0         0.0       0.0      0.0  
4      0.0      0.0       -13.0         0.0       0.0      0.0  

[5 rows x 196 columns]

In [4]:
len(df3) # 481187 (201612), 504444 (201712), 526677(201812), 542745(201909), 552658(202003)

552658

In [5]:
'eDDep' in df3.columns

True

In [6]:
# get mean eTotAst by IDRSSD:
df4 = df3[['IDRSSD','AsOf','Name','State','Cert','eTotAst','aTotAst','eTotLn','eDDep','eFDep']]
df4['eTotDep'] = df4['eDDep'] + df4['eFDep']
df5 = df4[df4['eTotAst']>0].groupby('IDRSSD')

In [7]:
df6x = pd.DataFrame(df5['AsOf'].max()).reset_index()
df6n = pd.DataFrame(df5['AsOf'].min()).reset_index()

In [8]:
df6n.tail()

IDRSSD       AsOf
10787  5398627 2020-03-31
10788  5401273 2019-09-30
10789  5412457 2019-09-30
10790  5427684 2019-12-31
10791  5448915 2019-12-31

In [9]:
df4.tail()

IDRSSD       AsOf                Name State     Cert  eTotAst  \
552653  5412457 2020-03-31        LEXICON BANK    NV  59117.0  58541.0   
552654  5427684 2019-12-31    LOYAL TRUST BANK    GA  59182.0  25679.0   
552655  5427684 2020-03-31    LOYAL TRUST BANK    GA  59182.0  30489.0   
552656  5448915 2019-12-31  MILLYARD BANK, THE    NH  59176.0  19968.0   
552657  5448915 2020-03-31  MILLYARD BANK, THE    NH  59176.0  23524.0   

        aTotAst   eTotLn    eDDep  eFDep  eTotDep  
552653  56230.0  36708.0  40537.0    0.0  40537.0  
552654  10998.0    941.0   1240.0    0.0   1240.0  
552655  29411.0  11372.0   7025.0    0.0   7025.0  
552656  11747.0    110.0    437.0    0.0    437.0  
552657  21076.0   6645.0   4504.0    0.0   4504.0

In [10]:
df7 = df6x.merge(df4[['IDRSSD','AsOf','Name','State','Cert']],how='inner', left_on=['IDRSSD','AsOf'], right_on=['IDRSSD','AsOf'])
df7.columns = ['IDRSSD', 'xAsOf', 'xName', 'xState','xCert']
df7.head()

IDRSSD      xAsOf                             xName xState    xCert
0      37 2020-03-31            BANK OF HANCOCK COUNTY     GA  10057.0
1     242 2020-03-31  FIRST COMMUNITY BANK XENIA-FLORA     IL   3850.0
2     279 2020-03-31       MINEOLA COMMUNITY BANK, SSB     TX  28868.0
3     354 2020-03-31                  BISON STATE BANK     KS  14083.0
4     439 2007-06-30                      PEOPLES BANK     GA  16498.0

In [11]:
df7 = df7.merge(df6n[['IDRSSD','AsOf']],how='inner', left_on=['IDRSSD'], right_on=['IDRSSD'])
df7.columns = ['IDRSSD', 'xAsOf', 'xName', 'xState','xCert', 'nAsOf']
df7.tail()

IDRSSD      xAsOf                xName xState    xCert      nAsOf
10787  5398627 2020-03-31  TRIAD BUSINESS BANK     NC  59188.0 2020-03-31
10788  5401273 2020-03-31          TANDEM BANK     GA  59178.0 2019-09-30
10789  5412457 2020-03-31         LEXICON BANK     NV  59117.0 2019-09-30
10790  5427684 2020-03-31     LOYAL TRUST BANK     GA  59182.0 2019-12-31
10791  5448915 2020-03-31   MILLYARD BANK, THE     NH  59176.0 2019-12-31

In [12]:
df5m = df5.mean()
df5m = df5m.reset_index()
df5m.drop('Cert', inplace=True, axis=1)
df8 = df7.merge(df5m, how='inner', left_on='IDRSSD', right_on='IDRSSD')

In [13]:
df8_orig = df8.copy()
len(df8) # 10762 (201612), 10767 (201712), 10775(201812), 10776(201903), 10787(201909), 10792(202003)

10792

In [14]:
df8.head()

IDRSSD      xAsOf                             xName xState    xCert  \
0      37 2020-03-31            BANK OF HANCOCK COUNTY     GA  10057.0   
1     242 2020-03-31  FIRST COMMUNITY BANK XENIA-FLORA     IL   3850.0   
2     279 2020-03-31       MINEOLA COMMUNITY BANK, SSB     TX  28868.0   
3     354 2020-03-31                  BISON STATE BANK     KS  14083.0   
4     439 2007-06-30                      PEOPLES BANK     GA  16498.0   

       nAsOf        eTotAst        aTotAst         eTotLn          eDDep  \
0 2001-03-31   80508.090909   80296.909091   38309.935065   64251.519481   
1 2001-03-31   34531.532468   34844.415584   21810.311688   26421.974026   
2 2001-03-31  161681.181818  155604.311688   83492.545455  113827.987013   
3 2001-03-31    9201.519481    9315.363636    4372.675325    8018.428571   
4 2001-03-31  166510.615385  165717.384615  118665.038462  144540.423077   

   eFDep        eTotDep  
0    0.0   64251.519481  
1    0.0   26421.974026  
2    0.0  113827.987013  
3    0.0    8018.428571  
4    0.0  144540.423077

In [15]:
df8.columns = ['IDRSSD','xAsOf','xName','xState','xCert','nAsOf','aeTotAst','aaTotAst','aeTotLn','aeDDep','aeFDep','aeTotDep']
df8.tail()

IDRSSD      xAsOf                xName xState    xCert      nAsOf  \
10787  5398627 2020-03-31  TRIAD BUSINESS BANK     NC  59188.0 2020-03-31   
10788  5401273 2020-03-31          TANDEM BANK     GA  59178.0 2019-09-30   
10789  5412457 2020-03-31         LEXICON BANK     NV  59117.0 2019-09-30   
10790  5427684 2020-03-31     LOYAL TRUST BANK     GA  59182.0 2019-12-31   
10791  5448915 2020-03-31   MILLYARD BANK, THE     NH  59176.0 2019-12-31   

           aeTotAst      aaTotAst       aeTotLn        aeDDep  aeFDep  \
10787  51525.000000  49079.000000      0.000000   3308.000000     0.0   
10788  33117.666667  28353.666667  10232.666667  13048.000000     0.0   
10789  44012.333333  36033.666667  21631.000000  25380.333333     0.0   
10790  28084.000000  20204.500000   6156.500000   4132.500000     0.0   
10791  21746.000000  16411.500000   3377.500000   2470.500000     0.0   

           aeTotDep  
10787   3308.000000  
10788  13048.000000  
10789  25380.333333  
10790   4132.500000  
10791   2470.500000

In [16]:
#############################
#############################
#  CAUTION: FailDate criterion here:
ff = pd.read_csv("CBfails_det202003.csv")

try:
  ## new colnames starting in 202003 csv file:
  ff2 = ff[['State', 'Year', 'NAME','CERT', 'FAILDATE']]  ## NOTE: Change in col names starting 202003
except:
  ## use next line (old colnames) for earlier versions of CB_fails_det201xxx.csv:
  ff2 = ff[['State', 'Year', 'Institution Name ','Cert ','Effective Date ']]

ff2.columns = ['State', 'Year', 'Name', 'Cert', 'EffDate']
ff2['FailDate'] = pd.to_datetime(ff2['EffDate'])
## ************************ ##
ff2 = ff2[ff2['FailDate']<='2020-03-31'] #  ONLY USE FAILDATES before cutoff date!!!!!!!!!!!! (to align with Call Data)
## ************************ ##
ff2.dropna(subset=['Cert'],inplace=True)
ff2['Cert'] = ff2['Cert'].astype(float)
ff2.head()

State  Year                              Name     Cert     EffDate  \
1    NE  2020                ERICSON STATE BANK  18265.0   2/14/2020   
2    NJ  2019  CITY NATIONAL BANK OF NEW JERSEY  21111.0   11/1/2019   
3    KY  2019             LOUISA COMMUNITY BANK  58112.0  10/25/2019   
4    OH  2019                     RESOLUTE BANK  58317.0  10/25/2019   
5    TX  2019                  ENLOE STATE BANK  10716.0   5/31/2019   

    FailDate  
1 2020-02-14  
2 2019-11-01  
3 2019-10-25  
4 2019-10-25  
5 2019-05-31

In [17]:
dff2 = ff2[ff2.duplicated(['Cert'])][['Name','Cert']]  ## find duplicate Certs
len(dff2)  # 44

44

In [18]:
ff2lst = list(dff2['Cert'])
len(ff2lst)                      ## list of dup Certs (28, now 44) (assistance, then failure)

44

In [19]:
ff2a = ff2[ff2.Cert.isin(ff2lst)]
ff2a.sort_index(by=['Cert','FailDate'],inplace=True)  ## sorted list of dup Certs
ff2a.head()

State  Year                              Name    Cert     EffDate  \
2372    TX  1988        FIRST CITY, TEXAS - DALLAS  1722.0   4/20/1988   
696     TX  1992    FIRST CITY, TEXAS - DALLAS, NA  1722.0  10/30/1992   
2373    TX  1988      FIRST CITY, TEXAS - BEAUMONT  3093.0   4/20/1988   
697     TX  1992  FIRST CITY, TEXAS - BEAUMONT, NA  3093.0  10/30/1992   
2374    TX  1988         FIRST CITY, TEXAS - BRYAN  3120.0   4/20/1988   

       FailDate  
2372 1988-04-20  
696  1992-10-30  
2373 1988-04-20  
697  1992-10-30  
2374 1988-04-20

In [20]:
## sections to dedup Cert field: (saves latest date, which usually is the fail date vs assistance date)

In [21]:
ff2x = ff2[['Cert','FailDate']].groupby('Cert').max()
ff2x.reset_index(drop=False,inplace=True)
ff2x['mx'] = 'x'
ff2x.tail()

Cert   FailDate mx
3564  90257.0 1992-08-21  x
3565  90266.0 1992-09-18  x
3566  90307.0 1992-08-14  x
3567  90310.0 1991-06-07  x
3568  91189.0 1998-07-23  x

In [22]:
ff3 = ff2.merge(ff2x,how='left',left_on=['Cert','FailDate'],right_on=['Cert','FailDate'])
ff3.tail()

State  Year                             Name     Cert    EffDate  \
3608    MO  1970  FIRST STATE BANK OF BONNE TERRE  12227.0  8/24/1970   
3609    NJ  1970          EATONTOWN NATIONAL BANK  19276.0   8/7/1970   
3610    KY  1970       FARMERS BANK OF PETERSBURG  13164.0  6/25/1970   
3611    MI  1970   THE PEOPLES STATE SAVINGS BANK  14987.0  4/18/1970   
3612    IA  1970       STATE BANK OF PRAIRIE CITY  14571.0  2/22/1970   

       FailDate mx  
3608 1970-08-24  x  
3609 1970-08-07  x  
3610 1970-06-25  x  
3611 1970-04-18  x  
3612 1970-02-22  x

In [23]:
ff4=ff3.dropna()
len(ff3), len(ff4) # 3613, 3569

(3613, 3569)

In [24]:
df8 = df8.merge(ff4[['Cert','FailDate']], how='left', left_on='xCert', right_on='Cert')
df8 = df8.drop('Cert', axis=1)
len(df8) # 10792

10792

In [25]:
df8.sort_index(by='aeTotAst',ascending=False, inplace=True)
df8.reset_index(drop=True, inplace=True)  # renumber rows based on largest to smallest banks
df8.reset_index(drop=False, inplace=True)  # move row numbers into another column called 'index'
df8.rename(columns={'index':'Rank'},inplace=True)  # rename 'index' column to 'Rank'
df8['Rank'] = df8['Rank'] + 1  # add 1 so that largest bank has rank 1 rather than rank 0
df8.head()

Rank  IDRSSD      xAsOf                    xName xState    xCert  \
0     1  852218 2020-03-31  JPMORGAN CHASE BANK, NA     OH    628.0   
1     2  480228 2020-03-31      BANK OF AMERICA, NA     NC   3510.0   
2     3  476810 2020-03-31           CITIBANK, N.A.     SD   7213.0   
3     4  451965 2020-03-31     WELLS FARGO BANK, NA     SD   3511.0   
4     5  484422 2009-12-31        WACHOVIA BANK, NA     NC  33869.0   

       nAsOf      aeTotAst      aaTotAst       aeTotLn        aeDDep  \
0 2001-03-31  1.542173e+09  1.516049e+09  5.482606e+08  7.277456e+08   
1 2001-03-31  1.307893e+09  1.308298e+09  6.849586e+08  8.495753e+08   
2 2001-03-31  1.090942e+09  1.086881e+09  5.184697e+08  3.028914e+08   
3 2001-03-31  9.457972e+08  9.340876e+08  5.642137e+08  6.660571e+08   
4 2001-03-31  4.416464e+08  4.372459e+08  2.546369e+08  2.704156e+08   

         aeFDep      aeTotDep   FailDate  
0  2.576001e+08  9.853457e+08        NaT  
1  1.043175e+08  9.538928e+08 2009-01-16  
2  4.486076e+08  7.514990e+08 2008-11-23  
3  6.632369e+07  7.323808e+08        NaT  
4  2.469325e+07  2.951088e+08        NaT

In [26]:
df8['pr_eTA'] = df8['aeTotAst']*100/df8['aeTotAst'].sum()
df8['pr_eTAcs'] = df8['pr_eTA'].cumsum()
df8['pr_eTL'] = df8['aeTotLn']*100/df8['aeTotLn'].sum()
df8['pr_eTLcs'] = df8['pr_eTL'].cumsum()
df8['pr_eDD'] = df8['aeDDep']*100/df8['aeTotLn'].sum()
df8['pr_eDDcs'] = df8['pr_eDD'].cumsum()
df8.head()

Rank  IDRSSD      xAsOf                    xName xState    xCert  \
0     1  852218 2020-03-31  JPMORGAN CHASE BANK, NA     OH    628.0   
1     2  480228 2020-03-31      BANK OF AMERICA, NA     NC   3510.0   
2     3  476810 2020-03-31           CITIBANK, N.A.     SD   7213.0   
3     4  451965 2020-03-31     WELLS FARGO BANK, NA     SD   3511.0   
4     5  484422 2009-12-31        WACHOVIA BANK, NA     NC  33869.0   

       nAsOf      aeTotAst      aaTotAst       aeTotLn        aeDDep  \
0 2001-03-31  1.542173e+09  1.516049e+09  5.482606e+08  7.277456e+08   
1 2001-03-31  1.307893e+09  1.308298e+09  6.849586e+08  8.495753e+08   
2 2001-03-31  1.090942e+09  1.086881e+09  5.184697e+08  3.028914e+08   
3 2001-03-31  9.457972e+08  9.340876e+08  5.642137e+08  6.660571e+08   
4 2001-03-31  4.416464e+08  4.372459e+08  2.546369e+08  2.704156e+08   

         aeFDep      aeTotDep   FailDate    pr_eTA   pr_eTAcs    pr_eTL  \
0  2.576001e+08  9.853457e+08        NaT  9.254091   9.254091  5.770519   
1  1.043175e+08  9.538928e+08 2009-01-16  7.848250  17.102340  7.209284   
2  4.486076e+08  7.514990e+08 2008-11-23  6.546401  23.648741  5.456966   
3  6.632369e+07  7.323808e+08        NaT  5.675430  29.324171  5.938428   
4  2.469325e+07  2.951088e+08        NaT  2.650180  31.974351  2.680089   

    pr_eTLcs    pr_eDD   pr_eDDcs  
0   5.770519  7.659623   7.659623  
1  12.979803  8.941898  16.601521  
2  18.436768  3.187974  19.789496  
3  24.375196  7.010344  26.799839  
4  27.055285  2.846162  29.646001

In [27]:
df8.to_csv("BkAvgAstPR_FailDate_202003.csv")

In [28]:
df8_fail = df8[df8['FailDate'].map(lambda x: x.year) > 2000]   #cbt[cbt['AsOf'].map(lambda x: x.month) == 12] 
len(df8_fail) # 518

518

In [29]:
df8_fail.to_csv("FailedBkList_AvgAst_202003.csv")